In [16]:
# Basic segmentation based on otsu
# Here the idea is to select sub-Roi and apply basic counting on each
# import sample data
import os
import skimage as ski
filename_Nuc = os.path.join(ski.data_dir, '/Users/zg/Mammouth/PROJECTS/Analysis/Image_Choice/MAX_PMLwt_20_MMStack_Pos0.ome-1-Nuc.tif')
Nucleus_PML = ski.io.imread(filename_Nuc)

filename_PML = os.path.join(ski.data_dir, '/Users/zg/Mammouth/PROJECTS/Analysis/Image_Choice/MAX_PMLwt_20_MMStack_Pos0.ome-1-PML.tif')
PML = ski.io.imread(filename_PML)


In [17]:
import napari

# create a `Viewer` and `Image` layer in Napari
viewer = napari.Viewer()
viewer.add_image(Nucleus_PML, colormap='blue', name='Nucleus PML')
viewer.add_image(PML, colormap='magma', name='PML')

<Image layer 'PML' at 0x163535760>

In [18]:
#segmentation using otsu
from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label
from skimage.morphology import closing, square, remove_small_objects

# apply threshold on PML bodies
thresh_PML = threshold_otsu(PML)
bw_PML = closing(PML > thresh_PML, square(5))

# remove artifacts connected to image border
cleared = remove_small_objects(clear_border(bw_PML), 10)

# label image regions
label_image_PML = label(cleared)

# add the labels
labels_layer = viewer.add_labels(label_image_PML, name='segmentation-PML')

In [19]:
# apply threshold on Nucleus
thresh_Nuc = threshold_otsu(Nucleus_PML)
bw_Nuc = closing(Nucleus_PML > thresh_Nuc, square(13))

# remove artifacts connected to image border
cleared = remove_small_objects(clear_border(bw_Nuc), 10)

# label image regions
label_image_Nuc = label(cleared)

# add the labels
labels_layer = viewer.add_labels(label_image_Nuc, name='segmentation-Nucleus')


In [20]:
Mask=viewer.layers[2].data

In [21]:
Result_PML=Mask*PML; #unclear if it is required
viewer.add_image(Result_PML)

<Image layer 'Result_PML' at 0x169e888c0>

In [24]:
#Here I want to extract only one of the selected label image from label_image_Nuc

import numpy as np

#Specify the label value of the ROI you want to extract
Mask_Nuc_label = 4  # the label value of your desired ROI

# Extract the ROI with the specified label value
roi_Mask_Nuc = np.where(label_image_Nuc == Mask_Nuc_label , 1, 0)
Result_Selected_roi_Nuc=Nucleus_PML*roi_Mask_Nuc;
# Display the extracted ROI using napari
viewer = napari.Viewer()
viewer.add_image(roi_Mask_Nuc, name='roi_Mask_Nuc')
viewer.add_image(Result_Selected_roi_Nuc, name='Result_Selected_roi_Nuc')
napari.run()

In [25]:
import numpy as np

#Specify the label value of the ROI you want to extract
Mask_PML_label = 4  # the label value of your desired ROI

# Extract the ROI with the specified label value
roi_Mask_PML = np.where(label_image_PML == Mask_PML_label , 1, 0)
Result_Selected_roi_PML=PML*roi_Mask_PML;
# Display the extracted ROI using napari
viewer.add_image(roi_Mask_PML, name='roi_Mask_PML')
viewer.add_image(Result_Selected_roi_PML, name='Result_Selected_roi_PML')

<Image layer 'Result_Selected_roi_PML' at 0x1677bbd10>

In [ ]:
#Particle analysis on PML on a sub-Roi called roi_Mask_Nuc
#from skimage import measure
#import pandas as pd
#
#region_properties_sub_roi_PML = measure.regionprops_table(roi_Mask_Nuc, PML, properties=['area', 'intensity_mean'])
#pd.DataFrame(region_properties_sub_roi_PML)


In [22]:
#Particle analysis on PML
from skimage import measure
import pandas as pd

#Result_PML or PML is enough ?
region_properties_PML = measure.regionprops_table(label_image_PML, Result_PML, properties=['area', 'intensity_mean'])
#pd.DataFrame(region_properties_PML);

In [ ]:
#Result_PML or PML is enough ?
region_properties_Nuc = measure.regionprops_table(label_image_Nuc, Nucleus_PML, properties=['area', 'intensity_mean'])
#pd.DataFrame(region_properties_Nuc);

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Convert the dictionaries to pandas DataFrames
regionNuc_df = pd.DataFrame(region_properties_Nuc)
regionPML_df = pd.DataFrame(region_properties_PML)

# Melt the DataFrames to reshape them for plotting
meltedNuc_df = regionNuc_df.melt(var_name='Property', value_name='Value')
meltedPML_df = regionPML_df.melt(var_name='Property', value_name='Value')

# Combine the melted DataFrames into a single DataFrame
combined_df = pd.concat([meltedNuc_df, meltedPML_df])

# Add a column to indicate the source of the data (Nucleus or PML)
combined_df['Source'] = ['Nucleus'] * len(meltedNuc_df) + ['PML'] * len(meltedPML_df)

# Create a violin plot with separate subplots for each property
plt.figure(figsize=(10, 6))
sns.violinplot(x='Source', y='Value', hue='Property', data=combined_df, split=True)

# Add labels and title
plt.xlabel('Source')
plt.ylabel('Value')
plt.title('Violin Plot of Region Properties')

plt.show()

In [ ]:
# Convert the dictionaries to pandas DataFrames
regionNuc_df = pd.DataFrame(region_properties_Nuc)
regionPML_df = pd.DataFrame(region_properties_PML)

# Create a scatter plot for the nucleus data
plt.figure(figsize=(10, 6))
sns.scatterplot(x='area', y='intensity_mean', data=regionNuc_df, label='Nucleus Size')
sns.scatterplot(x='area', y='intensity_mean', data=regionNuc_df, label='Nucleus Intensity')
plt.xlabel('Size')
plt.ylabel('Intensity')
plt.title('Scatter Plot of Nucleus Properties')
plt.legend()

# Create a scatter plot for the PML data
plt.figure(figsize=(10, 6))
sns.scatterplot(x='area', y='intensity_mean', data=regionPML_df, label='PML Size')
#sns.scatterplot(x='area', y='intensity_mean', data=regionPML_df, label='PML Intensity')
plt.xlabel('Size')
plt.ylabel('Intensity')
plt.title('Scatter Plot of PML Properties')
plt.legend()

plt.show()


In [ ]:
#Blob detection applied to PML bodies

from math import sqrt
from skimage import data
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray

import matplotlib.pyplot as plt



blobs_log = blob_log(PML, max_sigma=30, num_sigma=10, threshold=0.5)

# Compute radii in the 3rd column.
blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)

blobs_dog = blob_dog(PML, max_sigma=30, threshold=0.1)
blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)

blobs_doh = blob_doh(PML, max_sigma=30, threshold=0.01)

blobs_list = [blobs_log, blobs_dog, blobs_doh]
colors = ['yellow', 'lime', 'red']
titles = ['Laplacian of Gaussian', 'Difference of Gaussian', 'Determinant of Hessian']
sequence = zip(blobs_list, colors, titles)

fig, axes = plt.subplots(1, 3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

for idx, (blobs, color, title) in enumerate(sequence):
    ax[idx].set_title(title)
    ax[idx].imshow(PML)
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax[idx].add_patch(c)
    ax[idx].set_axis_off()

plt.tight_layout()
plt.show()